In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import numpy as np

mnist_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(mean=0.5, std=0.5)])

data = datasets.MNIST(root='/data/MNIST', download=True, transform=mnist_transforms)

mnist_dataloader = DataLoader(data, batch_size=64, shuffle=True, num_workers=0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /data/MNIST
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=0.5, std=0.5)
           )

In [3]:
test_data = iter(mnist_dataloader)
X = next(test_data)
print(len(X))
X[0].size()

2


torch.Size([64, 1, 28, 28])

In [4]:
X[0][0]

tensor([[[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1

In [6]:
class Generator(nn.Module):
    '''
    Generator class. Accepts a tensor of size 100 as input as outputs another
    tensor of size 784. Objective is to generate an output tensor that is
    indistinguishable from the real MNIST digits 
    '''

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_features=128, out_features=256),
                                    nn.LeakyReLU())
        self.layer2 = nn.Sequential(nn.Linear(in_features=256, out_features=512),
                                    nn.LeakyReLU())
        self.layer3 = nn.Sequential(nn.Linear(in_features=512, out_features=1024),
                                    nn.LeakyReLU())
        self.output = nn.Linear(in_features=1024, out_features=28 * 28)

        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.output(x)
        x = x.view(-1, 1, 28, 28)
        x = self.tanh(x)
        return x

In [7]:
class Discriminator(nn.Module):
    '''
    Discriminator class. Accepts a tensor of size 784 as input and outputs
    a tensor of size 1 as  the predicted class probabilities
    (generated or real data)
    '''

    def __init__(self):
        super().__init__()
        #self.hidden = nn.ModuleList()
        self.layer1 = nn.Sequential(nn.Linear(in_features=28 * 28, out_features=1024),
                                    nn.LeakyReLU())
        self.layer2 = nn.Sequential(nn.Linear(in_features=1024, out_features=512),
                                    nn.LeakyReLU())
        self.layer3 = nn.Sequential(nn.Linear(in_features=512, out_features=256),
                                    nn.LeakyReLU())
        self.output = nn.Sequential(nn.Linear(in_features=256, out_features=1),
                                    nn.Sigmoid())

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.output(x)
        return x

In [7]:
D = Discriminator().to(device)
G = Generator().to(device)
loss = nn.BCELoss()

D_optimizer = torch.optim.Adam(D.parameters(), lr=0.0001)
G_optimizer = torch.optim.Adam(G.parameters(), lr=0.0001)

for i in enumerate(mnist_dataloader):
    print(i[1][0].size())

In [8]:
for epoch in range(900):
    for idx, (imgs, _) in enumerate(mnist_dataloader):
        idx += 1
        # Обучаем дискриминатор
        # real_inputs - изображения из набора данных MNIST 
        # fake_inputs - изображения от генератора
        # real_inputs должны быть классифицированы как 1, а fake_inputs - как 0
        real_inputs = imgs.to(device)
        real_outputs = D(real_inputs)
        real_label = torch.ones(real_inputs.shape[0], 1).to(device)
        noise = (torch.rand(real_inputs.shape[0], 128) - 0.5) / 0.5
        noise = noise.to(device)
        fake_inputs = G(noise)
        fake_outputs = D(fake_inputs)
        fake_label = torch.zeros(fake_inputs.shape[0], 1).to(device)
        outputs = torch.cat((real_outputs, fake_outputs), 0)
        targets = torch.cat((real_label, fake_label), 0)
        D_loss = loss(outputs, targets)
        D_optimizer.zero_grad()
        D_loss.backward()
        D_optimizer.step()
        # Обучаем генератор
        # Цель генератора получить от дискриминатора 1 по всем изображениям
        noise = (torch.rand(real_inputs.shape[0], 128) - 0.5) / 0.5
        noise = noise.to(device)
        fake_inputs = G(noise)
        fake_outputs = D(fake_inputs)
        fake_targets = torch.ones([fake_inputs.shape[0], 1]).to(device)
        G_loss = loss(fake_outputs, fake_targets)
        G_optimizer.zero_grad()
        G_loss.backward()
        G_optimizer.step()
        #print(D_loss.item(), G_loss.item())
        if idx % 100 == 0 or idx == len(mnist_dataloader):
            print('Epoch {} Iteration {}: discriminator_loss {:.3f} generator_loss {:.3f}'.format(epoch, idx,
                                                                                                  D_loss.item(),
                                                                                                  G_loss.item()))
    if (epoch + 1) % 5 == 0:
        torch.save(G, './test/Generator_epoch_{}.pth'.format(epoch))
        print('Model saved.')

Epoch 0 Iteration 100: discriminator_loss 0.171 generator_loss 1.783
Epoch 0 Iteration 200: discriminator_loss 0.099 generator_loss 1.797
Epoch 0 Iteration 300: discriminator_loss 0.126 generator_loss 3.859
Epoch 0 Iteration 400: discriminator_loss 0.155 generator_loss 5.184
Epoch 0 Iteration 500: discriminator_loss 0.229 generator_loss 4.452
Epoch 0 Iteration 600: discriminator_loss 0.134 generator_loss 5.287
Epoch 0 Iteration 700: discriminator_loss 0.207 generator_loss 4.386
Epoch 0 Iteration 800: discriminator_loss 0.162 generator_loss 4.518
Epoch 0 Iteration 900: discriminator_loss 0.127 generator_loss 5.908
Epoch 0 Iteration 938: discriminator_loss 0.380 generator_loss 4.894
Epoch 1 Iteration 100: discriminator_loss 0.259 generator_loss 2.886
Epoch 1 Iteration 200: discriminator_loss 0.038 generator_loss 4.968
Epoch 1 Iteration 300: discriminator_loss 0.031 generator_loss 4.555
Epoch 1 Iteration 400: discriminator_loss 0.020 generator_loss 5.344
Epoch 1 Iteration 500: discriminat


KeyboardInterrupt



In [7]:
layer1 = nn.Sequential(
    nn.ConvTranspose2d(in_channels=100, out_channels=1024, stride=1, kernel_size=4, padding=0, bias=False),
    nn.BatchNorm2d(num_features=1024),
    nn.ReLU(inplace=True), )

layer2 = nn.Sequential(
    nn.ConvTranspose2d(in_channels=1024, out_channels=512, stride=2, kernel_size=4, padding=1, bias=False),
    nn.BatchNorm2d(num_features=512),
    nn.ReLU(inplace=True), )
layer3 = nn.Sequential(
    nn.ConvTranspose2d(in_channels=512, out_channels=256, stride=2, kernel_size=4, padding=1, bias=False),
    nn.BatchNorm2d(num_features=256),
    nn.ReLU(inplace=True), )
layer4 = nn.Sequential(
    nn.ConvTranspose2d(in_channels=256, out_channels=128, stride=2, kernel_size=4, padding=1, bias=False),
    nn.BatchNorm2d(num_features=128),
    nn.ReLU(inplace=True), )
layer5 = nn.Sequential(
    nn.ConvTranspose2d(in_channels=128, out_channels=3, stride=2, kernel_size=4, padding=1, bias=False),
    nn.Tanh())
x = torch.rand(10, 100)
x = x.reshape((x.shape[0], x.shape[1], 1, 1))
print(x.size())
print(layer1(x).size())
x = layer1(x)
print(layer2(x).size())
x = layer2(x)
print(layer3(x).size())
x = layer3(x)
print(layer4(x).size())
x = layer4(x)
print(layer5(x).size())
layer5(x)

torch.Size([10, 100, 1, 1])
torch.Size([10, 1024, 4, 4])
torch.Size([10, 512, 8, 8])
torch.Size([10, 256, 16, 16])
torch.Size([10, 128, 32, 32])
torch.Size([10, 3, 64, 64])


tensor([[[[-0.2782,  0.3445, -0.3528,  ...,  0.0174, -0.4197,  0.1874],
          [-0.4489,  0.3890,  0.3467,  ..., -0.0972,  0.7847, -0.0382],
          [-0.4859,  0.1005, -0.7718,  ...,  0.0491, -0.5042,  0.2689],
          ...,
          [-0.2143,  0.9263, -0.2161,  ...,  0.8581,  0.7893,  0.5876],
          [-0.5816, -0.2697,  0.2265,  ...,  0.2883, -0.6874,  0.4437],
          [ 0.0771,  0.2777, -0.0731,  ...,  0.3328,  0.0880, -0.0490]],

         [[-0.3202,  0.2165, -0.0047,  ...,  0.6146, -0.3220,  0.1655],
          [-0.4192,  0.0605,  0.2759,  ...,  0.7337,  0.1701,  0.3675],
          [-0.6238, -0.1057, -0.1238,  ..., -0.0664, -0.0596, -0.0920],
          ...,
          [-0.0018,  0.7566,  0.7461,  ...,  0.9128,  0.1981,  0.5471],
          [-0.2693,  0.2392, -0.6801,  ..., -0.5278, -0.5777,  0.1734],
          [-0.3589,  0.1306, -0.5616,  ...,  0.5361, -0.1228,  0.1405]],

         [[-0.1175,  0.2353, -0.1488,  ...,  0.2259, -0.1552,  0.2754],
          [ 0.4418,  0.2368,  

https://www.kaggle.com/code/vyacheslavshen/dcgan-pytorch-tutorial

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.clayer1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
                                     nn.LeakyReLU(),
                                     nn.MaxPool2d(kernel_size=2, stride=2)
                                     )
        self.clayer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
                                     nn.LeakyReLU(),
                                     nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(64 * 7 * 7, 1024)
        self.ac1 = torch.nn.LeakyReLU()
        self.fc2 = torch.nn.Linear(1024, 256)
        self.ac2 = torch.nn.LeakyReLU()
        self.fc3 = torch.nn.Linear(256, 10)
        self.ac_end = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.clayer1(x)
        x = self.clayer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.ac1(x)
        x = self.fc2(x)
        x = self.ac2(x)
        x = self.fc3(x)
        x = self.ac_end(x)
        return x

    def inference(self, x):
        x = self.forward(x)
        return nn.Softmax(x)

In [1]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 6))
noise = (torch.rand(1, 128) - 0.5) / 0.5
plt.tick_params(axis='both', length=0)
plt.axis("off")
plt.imshow(G((torch.rand(1, 128) - 0.5) / 0.5)[0].detach().numpy().transpose((1, 2, 0)))

NameError: name 'torch' is not defined

<Figure size 600x600 with 0 Axes>

In [79]:
model((torch.rand(1, 128) - 0.5) / 0.5)[0].detach().numpy().transpose((1, 2, 0))

array([[[-0.88548404],
        [-0.7913319 ],
        [-0.9817679 ],
        [-0.94669956],
        [-0.9718603 ],
        [-0.98320305],
        [-0.9967326 ],
        [-0.99200535],
        [-0.9570636 ],
        [ 0.7909233 ],
        [-0.9638806 ],
        [-0.97846156],
        [-0.97229147],
        [-0.60605186],
        [-0.84565663],
        [-0.98283106],
        [-0.9956229 ],
        [-0.7313035 ],
        [-0.93295574],
        [-0.82077456],
        [-0.7143531 ],
        [-0.89652544],
        [-0.95414615],
        [ 0.57141596],
        [-0.9575322 ],
        [-0.9831623 ],
        [-0.97540027],
        [-0.9885837 ]],

       [[-0.80315375],
        [-0.94764155],
        [-0.9790268 ],
        [-0.98374903],
        [-0.93665755],
        [-0.9983478 ],
        [-0.9332086 ],
        [-0.8807999 ],
        [ 0.82356393],
        [-0.8804912 ],
        [-0.93328255],
        [-0.9922101 ],
        [-0.58310944],
        [-0.9806069 ],
        [-0.99423444],
        [

In [44]:
noise

tensor([[ 0.5395,  0.9544,  0.6611, -0.4757, -0.8301,  0.8045,  0.3216,  0.9179,
         -0.6428,  0.1987,  0.1345,  0.5613, -0.2579,  0.2598,  0.3434, -0.4156,
         -0.6296,  0.0082, -0.6542,  0.1299,  0.3490, -0.7853, -0.0239, -0.6955,
         -0.8505, -0.9913,  0.8528, -0.1722, -0.3267, -0.1016, -0.2829,  0.4351,
          0.9579, -0.3798,  0.1456,  0.4221,  0.9750, -0.0136,  0.1775, -0.5544,
         -0.6090,  0.6258,  0.9881, -0.4932, -0.5739,  0.8516,  0.5848,  0.6350,
         -0.9634,  0.8150,  0.2418, -0.5369,  0.7791, -0.2261, -0.3193, -0.3820,
         -0.6224, -0.9535, -0.5765, -0.4264,  0.3109,  0.5459, -0.7324, -0.7304,
          0.8035, -0.5826, -0.6420,  0.7753,  0.5434,  0.6501, -0.6498, -0.3730,
          0.1997, -0.7406,  0.6460, -0.7222,  0.5282,  0.5178,  0.3215,  0.5724,
         -0.8679,  0.3225,  0.3004, -0.0300, -0.7594, -0.2622, -0.5610,  0.2544,
         -0.7090, -0.7260,  0.4633, -0.8133,  0.5436,  0.2079,  0.9787,  0.4213,
          0.4528,  0.3204,  

In [11]:
torch.rand(real_inputs.shape[0], 128)

tensor([[0.3784, 0.8643, 0.2690,  ..., 0.4714, 0.5403, 0.0324],
        [0.9346, 0.8003, 0.8330,  ..., 0.2683, 0.6323, 0.8304],
        [0.5615, 0.0870, 0.0309,  ..., 0.0019, 0.1108, 0.0951],
        ...,
        [0.1683, 0.0143, 0.8850,  ..., 0.4500, 0.0877, 0.6817],
        [0.7285, 0.1778, 0.6184,  ..., 0.5382, 0.1856, 0.6224],
        [0.9671, 0.6437, 0.0507,  ..., 0.0755, 0.1245, 0.8288]])

In [12]:
 (torch.rand(real_inputs.shape[0], 128) - 0.5) / 0.5

tensor([[-0.3078, -0.2757, -0.6496,  ...,  0.5296, -0.9710, -0.1025],
        [-0.4933, -0.1092, -0.4971,  ..., -0.4730,  0.9784, -0.5342],
        [ 0.1277,  0.2297,  0.0028,  ...,  0.0484,  0.7489,  0.5370],
        ...,
        [-0.2825,  0.4791, -0.6772,  ..., -0.6195, -0.6709,  0.2323],
        [ 0.1807, -0.8672,  0.5920,  ...,  0.9913,  0.1850,  0.7826],
        [-0.7598, -0.0346, -0.0097,  ..., -0.1969, -0.5449,  0.1601]])

In [113]:
import pytorch_lightning as pl


class GAN(pl.LightningModule):

    def __init__(self):
        super().__init__()
        # print(1)
        self.generator = Generator()
        self.discriminator = Discriminator()
        # After each epoch, we generate 100 images using the noise
        # vector here (self.test_noises). We save the output images
        # in a list (self.test_progression) for plotting later.
        self.test_noises = torch.randn(100, 1, 100, device=device)
        self.test_progression = []
        self.automatic_optimization = False

    def forward(self, z):
        """
        Generates an image using the generator
        given input noise z
        """
        #print('2')
        return self.generator(z)

    def generator_step(self, x):
        """
        Training step for generator
        1. Sample random noise
        2. Pass noise to generator to
           generate images
        3. Classify generated images using
           the discriminator
        4. Backprop loss to the generator
        """
        #print('3')

        # Sample noise
        z = torch.randn(x.shape[0], 1, 100, device=device)

        # Generate images
        generated_imgs = self(z)

        # Classify generated images
        # using the discriminator
        d_output = torch.squeeze(self.discriminator(generated_imgs))

        # Backprop loss. We want to maximize the discriminator's
        # loss, which is equivalent to minimizing the loss with the true
        # labels flipped (i.e. y_true=1 for fake images). We do this
        # as PyTorch can only minimize a function instead of maximizing
        g_loss = nn.BCELoss()(d_output,
                              torch.ones(x.shape[0], device=device))

        return g_loss

    def discriminator_step(self, x):
        #print('4')
        """
        Training step for discriminator
        1. Get actual images
        2. Predict probabilities of actual images and get BCE loss
        3. Get fake images from generator
        4. Predict probabilities of fake images and get BCE loss
        5. Combine loss from both and backprop loss to discriminator
        """

        # Real images
        d_output = torch.squeeze(self.discriminator(x))
        loss_real = nn.BCELoss()(d_output,
                                 torch.ones(x.shape[0], device=device))

        # Fake images
        z = torch.randn(x.shape[0], 1, 100, device=device)
        generated_imgs = self(z)
        d_output = torch.squeeze(self.discriminator(generated_imgs))
        loss_fake = nn.BCELoss()(d_output,
                                 torch.zeros(x.shape[0], device=device))

        return loss_real + loss_fake

    def training_step(self, batch, batch_idx):
        #print('5')
        X, _ = batch
        plt.imshow(np.reshape(X[0].detach().numpy(), (28, 28)))
        g_opt, d_opt = self.optimizers()

        g_loss = self.generator_step(X)
        g_opt.zero_grad()
        self.manual_backward(g_loss)
        g_opt.step()

        d_loss = self.discriminator_step(X)
        d_opt.zero_grad()
        self.manual_backward(d_loss)
        d_opt.step()

        #print(g_loss, d_loss)

    def configure_optimizers(self):
        #print('6')
        g_opt = torch.optim.Adam(self.generator.parameters(), lr=0.001)
        d_opt = torch.optim.Adam(self.discriminator.parameters(), lr=0.001)
        return g_opt, d_opt

    def on_train_epoch_end(self):  # , training_step_outputs
        #print('7')
        epoch_test_images = self(self.test_noises)
        self.test_progression.append(epoch_test_images)


#Теперь мы можем обучить наш GAN. Мы будем обучать его с помощью графического процессора в течение 100 эпох.

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = GAN()

trainer = pl.Trainer(max_epochs=2)  #, max_epochs=40,accelerator="auto"
trainer.fit(model, mnist_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 1.5 M 
1 | discriminator | Discriminator | 1.5 M 
------------------------------------------------
2.9 M     Trainable params
0         Non-trainable params
2.9 M     Total params
11.786    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

Error in callback <function flush_figures at 0x0000023C6465E700> (for post_execute):



KeyboardInterrupt



In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [114]:
import numpy as np
from matplotlib import pyplot as plt, gridspec

# Convert images from torch tensor to numpy array
images = [i.detach().cpu().numpy() for i in model.test_progression]

epoch_to_plot = 1
nrow = 4
ncol = 8

# randomly select 10 images for plotting
indexes = np.random.choice(range(100), nrow * ncol, replace=False)

fig = plt.figure(figsize=((ncol + 1) * 2, (nrow + 1) * 2))
fig.suptitle('Epoch {}'.format(epoch_to_plot), fontsize=30)

gs = gridspec.GridSpec(nrow, ncol,
                       wspace=0.0, hspace=0.0,
                       top=1. - 0.5 / (nrow + 1), bottom=0.5 / (nrow + 1),
                       left=0.5 / (ncol + 1), right=1 - 0.5 / (ncol + 1))

for i in range(nrow):
    for j in range(ncol):
        idx = i * ncol + j
        img = np.reshape(images[epoch_to_plot - 1][indexes[idx]], (28, 28))
        ax = plt.subplot(gs[i, j])
        ax.imshow(img, cmap='gray')
        ax.axis('off')

IndexError: list index out of range

<Figure size 1800x1000 with 0 Axes>

In [115]:
np.reshape(model.forward(torch.randn(1, 1, 100)).detach().numpy(), (28, 28))

array([[-1.        , -1.        , -1.        , -1.        ,  1.        ,
         1.        , -1.        , -1.        , -1.        , -1.        ,
         1.        , -1.        , -1.        ,  1.        ,  1.        ,
         1.        ,  1.        , -1.        , -1.        , -1.        ,
         1.        , -1.        , -1.        , -1.        ,  1.        ,
        -1.        , -1.        , -1.        ],
       [-1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
         1.        ,  1.        ,  1.        , -1.        ,  1.        ,
        -1.        ,  1.        ,  1.        , -1.        , -1.        ,
         1.        ,  1.        , -1.        ,  1.        , -1.        ,
        -1.        ,  1.        , -1.        ],
       [-1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        ,  1.        ,  1.        , -1.        ,
        -1.        , -1.    

In [57]:
img = np.reshape(model.forward(torch.randn(1, 1, 100)).detach().numpy(), (28, 28))
plt.imshow(img)

NameError: name 'model' is not defined

In [131]:
torch.randn(1, 1, 100).detach().numpy()

array([[[ 5.07983446e-01, -8.56134772e-01,  6.58681750e-01,
         -5.56978881e-01,  4.59508568e-01, -1.33551195e-01,
          1.58638585e+00, -3.67085814e-01,  7.08193839e-01,
          2.05746603e+00,  1.57912719e+00,  3.59650582e-01,
         -1.55611351e-01, -1.28941405e+00,  8.13412786e-01,
          3.04984629e-01,  1.17076226e-01, -7.31400728e-01,
          7.24779218e-02, -2.31657958e+00, -7.45728314e-01,
         -2.80001462e-01,  8.60050857e-01, -8.50371718e-01,
          1.25235513e-01,  1.00741494e+00, -2.04857901e-01,
         -1.15286922e+00,  4.12721694e-01,  1.02843940e+00,
          4.00168508e-01,  6.34475276e-02,  2.54313052e-01,
          1.00115073e+00, -8.31082985e-02,  1.27650928e+00,
          4.12714541e-01, -9.92530659e-02, -1.68523073e+00,
          2.28361294e-01,  2.34285280e-01,  1.47009119e-01,
          1.43849182e+00, -1.52908236e-01,  3.79862100e-01,
         -9.10960436e-01, -9.12463605e-01, -1.11210018e-01,
         -2.93070078e-01, -3.76423985e-0

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

mnist_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(mean=0.5, std=0.5),
                                       transforms.Lambda(lambda x: x.view(-1, 784))])

data = datasets.MNIST(root='/data/MNIST', download=True, transform=mnist_transforms)

mnist_dataloader = DataLoader(data, batch_size=32, shuffle=True, num_workers=0)


class Generator(nn.Module):
    '''
    Generator class. Accepts a tensor of size 100 as input as outputs another
    tensor of size 784. Objective is to generate an output tensor that is
    indistinguishable from the real MNIST digits 
    '''

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_features=100, out_features=256),
                                    nn.LeakyReLU())
        self.layer2 = nn.Sequential(nn.Linear(in_features=256, out_features=512),
                                    nn.LeakyReLU())
        self.layer3 = nn.Sequential(nn.Linear(in_features=512, out_features=1024),
                                    nn.LeakyReLU())
        self.output = nn.Sequential(nn.Linear(in_features=1024, out_features=28 * 28),
                                    nn.Tanh())

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.output(x)
        return x


class Discriminator(nn.Module):
    '''
    Discriminator class. Accepts a tensor of size 784 as input and outputs
    a tensor of size 1 as  the predicted class probabilities
    (generated or real data)
    '''

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_features=28 * 28, out_features=1024),
                                    nn.LeakyReLU())
        self.layer2 = nn.Sequential(nn.Linear(in_features=1024, out_features=512),
                                    nn.LeakyReLU())
        self.layer3 = nn.Sequential(nn.Linear(in_features=512, out_features=256),
                                    nn.LeakyReLU())
        self.output = nn.Sequential(nn.Linear(in_features=256, out_features=1),
                                    nn.Sigmoid())

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.output(x)
        return x

In [ ]:
import pytorch_lightning as pl


class GAN(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.generator = Generator()
        self.discriminator = Discriminator()
        # After each epoch, we generate 100 images using the noise
        # vector here (self.test_noises). We save the output images
        # in a list (self.test_progression) for plotting later.
        self.test_noises = torch.randn(100, 1, 100, device=device)
        self.test_progression = []

    def forward(self, z):
        """
        Generates an image using the generator
        given input noise z
        """
        return self.generator(z)

    def generator_step(self, x):
        """
        Training step for generator
        1. Sample random noise
        2. Pass noise to generator to
           generate images
        3. Classify generated images using
           the discriminator
        4. Backprop loss to the generator
        """

        # Sample noise
        z = torch.randn(x.shape[0], 1, 100, device=device)

        # Generate images
        generated_imgs = self(z)

        # Classify generated images
        # using the discriminator
        d_output = torch.squeeze(self.discriminator(generated_imgs))

        # Backprop loss. We want to maximize the discriminator's
        # loss, which is equivalent to minimizing the loss with the true
        # labels flipped (i.e. y_true=1 for fake images). We do this
        # as PyTorch can only minimize a function instead of maximizing
        g_loss = nn.BCELoss()(d_output,
                              torch.ones(x.shape[0], device=device))

        return g_loss

    def discriminator_step(self, x):
        """
        Training step for discriminator
        1. Get actual images
        2. Predict probabilities of actual images and get BCE loss
        3. Get fake images from generator
        4. Predict probabilities of fake images and get BCE loss
        5. Combine loss from both and backprop loss to discriminator
        """

        # Real images
        d_output = torch.squeeze(self.discriminator(x))
        loss_real = nn.BCELoss()(d_output,
                                 torch.ones(x.shape[0], device=device))

        # Fake images
        z = torch.randn(x.shape[0], 1, 100, device=device)
        generated_imgs = self(z)
        d_output = torch.squeeze(self.discriminator(generated_imgs))
        loss_fake = nn.BCELoss()(d_output,
                                 torch.zeros(x.shape[0], device=device))

        return loss_real + loss_fake

    def training_step(self, batch, batch_idx, optimizer_idx):
        X, _ = batch

        # train generator
        if optimizer_idx == 0:
            loss = self.generator_step(X)

        # train discriminator
        if optimizer_idx == 1:
            loss = self.discriminator_step(X)

        return loss

    def configure_optimizers(self):
        g_optimizer = torch.optim.Adam(self.generator.parameters(), lr=0.0002)
        d_optimizer = torch.optim.Adam(self.discriminator.parameters(), lr=0.0002)
        return [g_optimizer, d_optimizer], []

    def on_train_epoch_end(self, training_step_outputs):
        epoch_test_images = self(self.test_noises)
        self.test_progression.append(epoch_test_images)


#Теперь мы можем обучить наш GAN. Мы будем обучать его с помощью графического процессора в течение 100 эпох.

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = GAN()

trainer = pl.Trainer(max_epochs=100)
trainer.fit(model, mnist_dataloader)

In [ ]:
for i in torch.manual_seed(42):
    print(i)